In [1]:
#import tensorflow as tf
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
import numba as nb
import sys
import openseespy.opensees as osp

In [2]:
@nb.vectorize(["float64(int64,int64,int64,float64)"], target='parallel')
def coordX(M, vx, vy, dx):
    if M <  ( (vx+1) * (vy+1) ):
        return  (M % (vx+1) ) * (dx)
    else:
        return  ( ( M - (vx+1) * (vy+1) ) % (vx*2 +1) ) * (dx/2) 

In [3]:
@nb.vectorize(["float64(int64,int64,int64,float64)"], target='parallel')
def coordY(M, vx, vy, dy):
    if M <  ( (vx+1)* (vy+1) ):
        return  ((M % ( (vx+1) * (vy+1) )// (vx+1) ) ) * (dy)
    else:
        return  ( ((M - (vx+1) * (vy+1) ) % ( (vx*2 +1) * (vy*2 +1) ))// (vx*2 +1) ) * (dy/2)

In [4]:
@nb.vectorize(["float64(int64,int64,int64,float64)"], target='parallel')
def coordZ(M, vx, vy, dz):
    if M < ( (vx+1)* (vy+1) ):
        return  0.0
    else:
        return  (((M - ( (vx+1)* (vy+1) ) ) // ( (vx*2 +1)*(vy*2 +1))) + 1) * (dz)

In [5]:
@nb.jit(["void(int32,float64,float64,float64)"], parallel = True)
def ospyNode(M, cx, cy, cz):
    for i in nb.prange(M.shape[0]):
        osp.node(int(M[i]),cx[i],cy[i],cz[i])
        if cz[i] == 0.0:
            osp.fix(int(M[i]), 1, 1, 1, 1, 1, 1)

In [6]:
@nb.jit(["void(int64,float64,int64,float64,int64,float64)"])
def estruct(vx, dx, vy, dy, vz, dz):
    # numero total de pontos
    Npts = (vx*2 +1) * (vy*2 +1) * (vz) + ( (vx+1) * (vy+1) )
    # Matriz dos indices
    mat = np.arange(0,Npts, 1, dtype=np.int64 )
    # corrdenadas dos pontos
    cX = coordX(mat, vx, vy, dx)
    cY = coordY(mat, vx, vy, dy)
    cZ = coordZ(mat, vx, vy, dz)
    #print( '\ncoord X\n', cX , '\nCoord Y\n', cY, '\nCoord Z\n', cZ)
    # matriz dos andares superiores
    matA = mat[(vX+1)*(vY+1)::].reshape(vZ, vY*2 +1, vX*2 +1)
    # matriz dos indices iniciais das vigas em x
    matXI = matA[::,::2,:-1:]
    # matriz dos indides finais das vigas em x
    matXF = matA[::,::2,1::]
    # matriz dos indices inicias das vigas em Y
    matYI = matA[::,:-1:,::2]
    # matriz dos indices finais das vigas em Y
    matYF = matA[::,1::,::2]
    # matriz pontos da fundação
    matB = mat[: (vx+1) * (vy+1)]
    # matriz final dos pilares
    matPF = matA[::,::2,::2]
    # matriz inicial dos pilares
    matPI = np.append(matB,matPF[:-1:,::,::]).reshape(vZ,(vY+1),(vX+1))
    #print("matriz andades\n", matA)
    # Matrizes de superficie 
    matBL = matA[::, :-1:, :-1:]
    #print("Inferior esquerdo\n",matBL)
    matBR = matA[::, :-1:, 1::]
    #print("Inferior direito\n",matBR)
    matTL= matA[::, 1::, :-1:]
    #print("Superior esquerdo\n",matTL)
    matTR = matA[::, 1::, 1::]
    #print("Superior direito\n",matTR)
    
    # - Modelo Openseespy
    osp.wipe()
    # init model
    # degrees of freedom by node
    ndf=6
    osp.model('basic', '-ndm', 3, '-ndf', ndf)
    ospyNode(mat, cX, cY, cZ)    

In [7]:
# entradas
# numero de vaos em x
vX = 2
# distancia dos vaos em x
dX = 5.25
# número de vãos em y
vY = 3
# distancia dos vaos em y
dY = 7.25
# numero pavimentos
vZ = 2
 # distancia piso a piso
dZ = 3.25

In [8]:
%%timeit
estruct(vX, dX, vY, dY, vZ, dZ)

1.34 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
teste = osp.nodeCoord(40)
print(teste)

[7.875, 18.125, 3.25]
